In [1]:
import keras
import os
import pickle
import cv2 as cv2
from sklearn import preprocessing
import sklearn.preprocessing
import numpy as np
from os import walk
import matplotlib.pyplot as plt
from PIL import Image, ImageOps;
from keras.models import Model
from keras.preprocessing import image
from keras.applications import VGG16
from keras.applications.densenet import DenseNet201
from keras.applications.densenet import preprocess_input
from keras import models
from keras import layers
from keras.models import load_model
seed = 13
np.random.seed(seed)


Using TensorFlow backend.


In [2]:
conv_base = VGG16(weights='imagenet',
                  include_top=True,
                  input_shape=(224, 224, 3))

In [3]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
from skimage.io import imread

import imgaug as ia
from imgaug import augmenters as iaa

In [5]:
#Creo una función para data aumentation
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips con una prob del 50%
    iaa.Flipud(0.5), # vertical flips con una prob del 50%
    # Small gaussian blur with random sigma between 0 and 0.5.
    # Hacemos un desenfoque a las imagenes pero solo un 50% del dataset approx, 
    # el valor del desonfoque es aleatorio entre 0-0.75
    iaa.Sometimes(0.5,
                  iaa.GaussianBlur(sigma=(0, 0.75))
                 ),
    
    # Modificamos el contraste de forma random
    iaa.ContrastNormalization((0.1, 1.5)),
    # añadimos ruido gausiano de forma aleatoria
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),

    # hacemos las imagenes un poco mas brillantes o oscuras de forma aleatoria
    # puede llegar a cambiar el color
    
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # aplicamos una transformacion affine, que cambia la escala, el zoom rotacion
    # translacion. Solo a un 50% del dataset
    iaa.Sometimes(0.5,
                     iaa.Affine(
                     scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                     translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                     rotate=(-25, 25),
                     shear=(-8, 8)
    )
                 )

], random_order=True)

In [6]:
#Esta es una de las maneras en las que se puede jugar con los modelos preentrenados
#from keras import models
#from keras import layers
#from keras.layers import Dense
#n_classes = 13
#fc2 = conv_base.get_layer('fc2').output
#mypredictions = Dense(n_classes, activation='softmax', name='mypredictions')(fc2)
#model = Model(input =conv_base.input, output =mypredictions)
#keras.layers.Dropout(0.5,seed)

In [7]:
from keras import models
from keras import layers 

fc1 = conv_base.get_layer('fc1').output

Drop1 = layers.Dropout(0.8, seed = seed)(fc1) #Esto evitará el overfiting
Dense1 = layers.Dense(4096, activation='relu',name='fc2')(Drop1)
Drop2 = layers.Dropout(0.8, seed = seed)(Dense1)#Esto evitará el overfiting
Dense2 = layers.Dense(13, activation='sigmoid',name='mypredictions')(Drop2)

model = Model(inputs=conv_base.input, outputs = Dense2)

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [12]:
#from keras import models      #4096
#from keras import layers

#model = models.Sequential()
#model.add(conv_base)
#model.add(layers.Dense(256, activation='relu'))
#model.add(layers.Dense(13, activation='sigmoid'))

In [14]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      #rescale=1./255,
      rotation_range=50,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      brightness_range=(0.2,1.5),
      preprocessing_function=seq.augment_image,
      fill_mode='nearest')

train_dir ="./dataset"
validation_dir= "./dataset_test"
# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(
    #rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        target_size=(224, 224),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='categorical')


Found 1911 images belonging to 13 classes.
Found 477 images belonging to 13 classes.


In [15]:
print(train_generator.class_indices)

{'mercat_independencia': 9, 'societat_general': 11, 'desconegut': 3, 'farmacia_albinyana': 7, 'ajuntament': 0, 'mnactec': 10, 'escola_enginyeria': 5, 'masia_freixa': 8, 'castell_cartoixa': 1, 'dona_treballadora': 4, 'catedral': 2, 'teatre_principal': 12, 'estacio_nord': 6}


In [16]:
print(validation_generator.class_indices)

{'mercat_independencia': 9, 'societat_general': 11, 'desconegut': 3, 'farmacia_albinyana': 7, 'ajuntament': 0, 'mnactec': 10, 'escola_enginyeria': 5, 'masia_freixa': 8, 'castell_cartoixa': 1, 'dona_treballadora': 4, 'catedral': 2, 'teatre_principal': 12, 'estacio_nord': 6}


In [17]:
model.trainable = True



In [18]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5),
              metrics=['acc'])

In [19]:
model.layers[1].trainable

True

In [20]:
#Genero un callback, consiste en que se guarda la mejor época
filepath ='Nombre_del_modelo.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
callbacks_list = [checkpoint]

In [26]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=1194//20,
      epochs=50,
      callbacks=callbacks_list,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/50
59/59 [==============================] - 60s 1s/step - loss: 0.2650 - acc: 0.9230 - val_loss: 0.3578 - val_acc: 0.9356

Epoch 00001: val_acc did not improve from 0.94467
Epoch 2/50
59/59 [==============================] - 59s 993ms/step - loss: 0.2525 - acc: 0.9356 - val_loss: 0.3150 - val_acc: 0.9497

Epoch 00002: val_acc improved from 0.94467 to 0.94970, saving model to VGG_Dropout+Efectos.h5
Epoch 3/50
59/59 [==============================] - 59s 997ms/step - loss: 0.2131 - acc: 0.9390 - val_loss: 0.3810 - val_acc: 0.9384

Epoch 00003: val_acc did not improve from 0.94970
Epoch 4/50
59/59 [==============================] - 59s 993ms/step - loss: 0.1951 - acc: 0.9491 - val_loss: 0.3224 - val_acc: 0.9416

Epoch 00004: val_acc did not improve from 0.94970
Epoch 5/50
59/59 [==============================] - 59s 997ms/step - loss: 0.2427 - acc: 0.9288 - val_loss: 0.3099 - val_acc: 0.9366

Epoch 00005: val_acc did not improve from 0.94970
Epoch 6/50
59/59 [=====================

In [25]:
#preprocessing_function=seq.apply_image

In [ ]:
#model.save('8_Entrega_kaggle_3.h5')
#np.save('8_Entrega_kaggle_3', history.history)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
import matplotlib.pyplot as plt


epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [15]:
clases = {}
clases[0] = 'ajuntament'
clases[1] = 'castell_cartoixa'
clases[2] = 'catedral'
clases[3] = 'dona_treballadora'
clases[4] = 'escola_enginyeria'
clases[5] = 'estacio_nord'
clases[6] = 'farmacia_albinyana'
clases[7] = 'masia_freixa'
clases[8] = 'mercat_independencia'
clases[9] = 'mnactec'
clases[10] = 'societat_general'
clases[11] = 'teatre_principal'
#dictio_matrix['desconegut'] = 12

In [16]:
clases

{0: 'ajuntament',
 1: 'castell_cartoixa',
 2: 'catedral',
 3: 'dona_treballadora',
 4: 'escola_enginyeria',
 5: 'estacio_nord',
 6: 'farmacia_albinyana',
 7: 'masia_freixa',
 8: 'mercat_independencia',
 9: 'mnactec',
 10: 'societat_general',
 11: 'teatre_principal'}